### Initialization

In [2]:
"""IMPORTS"""

from arcgis.gis import GIS
from arcgis.gis import Item
from arcgis.gis import ContentManager
from arcgis import __version__
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from io import BytesIO
import ipywidgets as widgets

import pandas as pd
import tempfile

import os
import uuid
import json
import shutil
import tempfile

from getpass import getpass

In [27]:
"""CONSTANTS""" 

CATALOG_ID = "3f48c8811d5b4298a9b27da22b00bfd6"
ORIGIN_TRANSFER_USER = "transfer_inscomm"
DESTINATION_TRANSFER_USER = "rya12533@esri.com_fsi"

### DO NOT CHANGE

ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url']

TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene', 'Dashboard',
                                   'Image Service', 'Feature Collection', 
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application', 
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration'])

FILE_BASED_ITEM_TYPES = frozenset(['File Geodatabase','CSV', 'Image', 'KML', 'Locator Package',
                                  'Map Document', 'Shapefile', 'Microsoft Word', 'PDF',
                                  'Microsoft Powerpoint', 'Microsoft Excel', 'Layer Package',
                                  'Mobile Map Package', 'Geoprocessing Package', 'Scene Package',
                                  'Tile Package', 'Vector Tile Package'])

RELATIONSHIP_TYPES = frozenset(['Map2Service', 'WMA2Code',
                                'Map2FeatureCollection', 'MobileApp2Code', 'Service2Data',
                                'Service2Service'])

_version = [int(i) for i in __version__.split('.')]


### CONFIGURE HERE

In [30]:
### HARDCODED CONFIG FOR TESTING SM TRANSFER

origin_pass = getpass(prompt=f"Enter the password for user {ORIGIN_TRANSFER_USER}: ")

# Establish origin and target GIS organizations
print("Connecting ...")
origin = GIS("https://inscomm.maps.arcgis.com/", ORIGIN_TRANSFER_USER, origin_pass, expiration=9999)
print("Connection Successful.")

destination = GIS("https://fsi.maps.arcgis.com/", client_id="U4Aj4BTsHvPho3aa", expiration=9999)

Connecting ...
Connection Successful.
Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://fsi.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=U4Aj4BTsHvPho3aa&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=2yRHjqmgdijdgvIo5FhyfQtVJG3X1J&allow_verification=false


c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fsi.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [31]:
catalog = destination.content.get(CATALOG_ID)
catalog_table = pd.DataFrame.spatial.from_layer(catalog.tables[0])

text_box = widgets.Text(description="Please input the Item ID of the item you would like to transfer below: ")
display(text_box)

Text(value='', description='Please input the Item ID of the item you would like to transfer below: ')

### Helper Functions

In [10]:
def rebuild_relations(destination: GIS, relation_map: dict):
    """
    Helper Function
    
    Applies origin item relations to the destination portal item.
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the origin item.
        relation_map (dict): Relation map returned from wc_transfer() that maps the origin and destination IDs. 
    """
    
    # traverse all keys in the relation map, representing the source item IDs
    for key in relation_map.keys():
        origin_item = origin_items_by_id[key]
        destination_itemid = relation_map[key]
        destination_item = destination.content.get(destination_itemid)
        
        # find the relations attatched to this ID 
        for relationship in RELATIONSHIP_TYPES:
            try:
                origin_related_items = origin_item.related_items(relationship)
                
                # apply relationships to target items 
                for origin_related_item in origin_related_items:
                    destination_related_itemid = relation_map[origin_related_item.itemid]
                    destination_related_item = destination.content.get(destination_related_itemid)
                    status = destination_item.add_relationship(destination_related_item, relationship)
                    print(f"After execution, relation type {status} between {destination_item.title} and {destination_related_item.title} is {status}.")
            except Exception as rel_ex: # bare except clause temporary
                print(f"Error when checking for {relationship}: {rel_ex}")
                continue
    
    print("Process complete. Please check your content on the destination portal and review errors raised in this notebook.")
    
def export_resources(item, save_path=None, file_name=None):
    """
    Helper function, from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    Export's the data's resources as a zip file
    """
    
    url = f'{item._gis._portal.resturl}content/users/{item._user_id}/items/{item.itemid}/resources/export'
    if save_path is None:
        save_path = tempfile.gettempdir()
    if file_name is None:
        file_name = f"{uuid.uuid4().hex[:6]}.zip"
    params = {'f' : 'zip'}
    con = item._gis._portal.con
    resources = con.get(url, params=params,
                        out_folder=save_path,
                        file_name=file_name,
                        try_json=False)
    return resources

def get_layer_item_ids(wm):
    """
    Helper function from https://developers.arcgis.com/python/guide/cloning-content/
    
    Returns the related items in a webmap.
    
    Params:
        wm (argis.gis.Item): Webmap item to be inspected.
    Returns:
        wm_id_list (list): List of related items in the web map.
    """
    wmo = WebMap(wm)
    wm_id_list = []
    
    for layer in wmo.layers:
        try:
            fsvc = FeatureLayerCollection(layer['url'][:-1], origin)
            if not fsvc.properties['serviceItemId'] in wm_id_list:
                wm_id_list.append(fsvc.properties['serviceItemId'])
        except Exception as e:
            pass
    return wm_id_list

def iterate_all(iterable, returned="key"):
    # Credits: https://gist.github.com/PatrikHlobil/9d045e43fe44df2d5fd8b570f9fd78cc
    
    """Returns an iterator that returns all keys or values
       of a (nested) iterable.
       
       Arguments:
           - iterable: <list> or <dictionary>
           - returned: <string> "key" or "value"
           
       Returns:
           - <iterator>
    """
  
    if isinstance(iterable, dict):
        for key, value in iterable.items():
            if returned == "key":
                yield key
            elif returned == "value":
                if not (isinstance(value, dict) or isinstance(value, list)):
                    yield value
            else:
                raise ValueError("'returned' keyword only accepts 'key' or 'value'.")
            for ret in iterate_all(value, returned=returned):
                yield ret
    elif isinstance(iterable, list):
        for el in iterable:
            for ret in iterate_all(el, returned=returned):
                yield ret
                
def find_relates(item: Item, gis=GIS("home")):
    """
    Finds the AGOL items nested inside an item without needing an Enterprise platform or 
    manually specifying relationships using the ArcGIS API. Does not tell you the type of 
    relationship between items, only the ItemIDs associated with a given item.
    
    Arguments:
        gis (GIS): GIS object from arcgis.gis.GIS that the item lives in.
        item (Item): Item object from arcgis.gis.Item
        
    Returns:
        related_ids (set): All unique item IDs related to an item.
    """
    
    related_ids = []
    item_json = item.get_data(try_json=True)
    formatted_keys = iterate_all(item_json, returned="value")
    
    json_vals = list(formatted_keys)
    json_vals = [val for val in json_vals if isinstance(val, str)]
    
    for val in json_vals:
        try:
            search = gis.content.get(val)
            if search != None:
                related_ids.append(val)
        except:
            pass
    
    return set(related_ids)

def get_dash_wm(dash):
    """
    From https://developers.arcgis.com/python/guide/cloning-content/#helper-functions
    
    Returns a list of all Web Maps participating in a Dashboard. 
    
    Arguments:
        dash (item): Dashboard to return participating Web Maps from.
    Returns:
        (list): All Web Maps partipating in the dashboard. 
    
    """
    return [origin.content.get(widget['itemId']) 
            for widget in dash.get_data()["desktopView"]['widgets']
            if widget['type'] == "mapWidget"]

### Transfer Functions

In [36]:
def wc_transfer(destination: GIS, items=[]):
    """
    Performs a web content transfer of items from an origin to destination AGOL. 
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the given items
        items (list): a list of Items to be transferred.
    """

    origin_to_destination_ids = {}
    
    # check if an item has been transferred. if it has, remove it from the clone list.
    for item in items:
        if item.id in catalog_table['source_id'].unique():
            items.remove(item)
            print(f"{item.title} already transferred. Removing it from the queue...")
    
    destination_items = destination.content.clone_items(items)
    
    # build origin to destination map for rebuild_relations function and to memoize to catalog
    origin_item_index = 0
    for destination_item in destination_items:
        if destination_item:
            origin_to_destination_ids[item.id] = destination_item.id # TODO
        else:
            origin_to_destination_ids[item.id] = None
            
        new_record = pd.DataFrame({"source_id": items[origin_item_index].id,
                           "destination_id": destination_item.id,
                           "title": destination_item.title,
                           "owner": destination_item.owner})
        pd.concat([catalog_table, new_record])
        origin_item_index += 1
    
    # export to csv in memory, and overwrite current catalog feature layer
    catalog_table.to_csv('catalog.csv')
    
    collection = FeatureLayerCollection.fromitem(catalog)
    collection.manager.overwrite('catalog.csv')
            
    # run rebuild helper function
    rebuild_relations(destination=destination, relation_map=origin_to_destination_ids)
    
def dash_transfer(destination: GIS, dash):
    """    
    Performs a web content transfer of a dashboard to destination AGOL.
    
    Arguments:
        destination (GIS): Destination GIS for the given Dashboard
        dash (Item): A Dashboard item in the origin GIS
    """

    wm_items = {}
    dash_elements = get_dash_wm(dash=dash)
    
    for ele in dash_elements:
        # check if the item has been transferred already
        if ele.id in catalog_table['source_id'].unique():
            try:
                ele_dest_id  = catalog_table.loc[catalog_table['source_id'] == ele.id, 'destination_id'].values[0]
                wm_items[ele] = ele_dest_id
            except IndexError:
                continue
        # if the item participating in the dashboard has not yet been cloned: 
        else:
            print(f"Item {ele.title} has not been generated in the target org. Building now...")
            wc_transfer(destination=destination, items=[ele])

    # transfer the dashboard
    destination.content.clone_items(items=[dash], item_mapping=wm_items)
    
    # swizzle the old item IDs with the new ones
    # TODO : Only relevant if working on Arcgis API 2.2.0 release.  
    
    print(f"Clone successful. Refresh your content page.")

def sm_transfer(destination: GIS, item: Item):
    """
    Adapted code sample from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    
    Transfer protocol for Story Maps and their web content items. Does not call copy_items() as protocol is different for this content.
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the origin item.
    """

    if item.id in catalog_table['source_id'].unique():
        print(f"Story Map with ID {item.id} has already been transferred. Moving to next portal item.")
        pass
    story_map = item

    # check version to apply relevant protocol
    if _version <= [1, 8, 2]:
        resource = export_resources(item=story_map)
    else:
        resource = story_map.resources.export()

    # get story map item data from json to store related maps
    story_map_json = story_map.get_data(try_json=True)

    web_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('webmap')>-1])
    express_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('expressmap')>-1])


    webmap_mapper = {} # keys are origin IDs, values are destination IDs
    for wm in web_maps:
        webmap_to_copy = origin.content.get(wm)

        # check if item has been tranferred, if no then duplicate, if yes point to content that already exists
        if webmap_to_copy.id in catalog_table['source_id'].unique():
            webmap_destination_id = catalog_table.loc[catalog_table['source_id'] == webmap_to_copy.id, 'destination_id']
            webmap_mapper[webmap_to_copy.id] = webmap_destination_id
        else:
            cloned_webmaps = destination.content.clone_items([webmap_to_copy])
            webmap_mapper[webmap_to_copy.id] = [i for i in cloned_webmaps if i.type == 'Web Map'][0].id

            # memoize tranfer to catalog
            new_record = pd.DataFrame({"source_id": [webmap_to_copy.id],
                                       "destination_id": [webmap_mapper[webmap_to_copy.id]], # gets destination ID from corresponding origin ID
                                       "title": [webmap_to_copy.title],
                                       "owner": [webmap_to_copy.owner]})
            catalog_table = pd.concat([catalog_table, new_record])
    
    # export to csv in memory, and overwrite current catalog feature layer
    catalog_table.to_csv('catalog.csv')
    
    collection = FeatureLayerCollection.fromitem(catalog)
    collection.manager.overwrite('catalog.csv')

    # remap the old itemid to the new one
    story_map_text = json.dumps(story_map_json)

    for key, val in webmap_mapper.items():
        story_map_text = story_map_text.replace(key, val)

    new_item = destination.content.add({'type' : story_map.type,
                             'tags' : story_map.tags,
                             'title' : story_map.title,
                             'description' : story_map.description,
                             'typeKeywords' : story_map.typeKeywords,
                             'extent' : story_map.extent,
                             'text' :story_map_text}
                            )

    # bring in the storymap resources exported to a zip archive earlier
    new_item.resources.add(resource, archive=True)

    # update the url
    new_item.update({'url': story_map.url.replace(story_map.id, new_item.id)})
    print(f"StoryMap transfer complete. You can visit it at {new_item.homepage}")

def transfer(item=text_box.value):
    """
    Runs the appropriate transfer function for each item the user has chosen from the select widget.
    
    Arguments:
        items (iterable): An iterable of Item IDs. Default is the set generated from the selection box in the Content Migration Notebook.
    """
    item_origin = origin.content.get(item)
    
    # reassign item to origin transfer user
    if item_origin.owner != ORIGIN_TRANSFER_USER:
        item_origin.reasign_to(ORIGIN_TRANSFER_USER)
    
    # decide on appropriate workflow for item
    if item_origin.type == "StoryMap":
        sm_transfer(destination=destination, item=item_origin)
    elif item_origin.type == "Dashboard":
        dash_transfer(destination=destination, dash=item_origin)
    else:
        wc_transfer(destination=destination, items=[item_origin])
    


In [40]:
dir(Item)

['_RELATIONSHIP_DIRECTIONS',
 '_RELATIONSHIP_TYPES',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_publish_status',
 '_create_proxy',
 '_delete_proxy',
 '_get_icon',
 '_get_nbs_server',
 '_has_layers',
 '_hydrate',
 '_is_notebook',
 '_populate_layers',
 '_repr_html_',
 '_snapeshots',
 '_uid',
 '_user_id',
 '_ux_item_type',
 '_validate_url',
 'add_comment',
 'add_relationship',
 'app_info',
 'can_delete',
 'clear',
 'comments',
 'content_status',
 'copy',
 'copy_feature_layer_collect

In [38]:
transfer()

Item Boston Policies has not been generated in the target org. Building now...


_ItemCreateException: ("Failed to create Feature Service Houses: A connection error has occurred: HTTPSConnectionPool(host='fsi.maps.arcgis.comsharing', port=443): Max retries exceeded with url: /rest/search (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000189B9515C40>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))", None)